In [ ]:
install.packages("languageserver")
library(ggplot2)
library(dplyr)

---

**Scenario**: A salesman has scheduled two appointments to sell software, one in the morning and another one in the afternoon. There are two software
editions available: the base edition costing Rs. 5000 and the premium edition costing Rs. 10000. His morning appointments typically lead
to a sale with a 30% chance while the afternoon ones typically lead to a sale with a 60% chance independent of what happened in the
morning. If the morning appointment ends up in sale, the salesman has a 70% chance of selling the premium edition and if the afternoon
appointment ends up in a sale, he is equally likely to sell either of the editions. Let X be the random variable representing the total Rupee
value of sales. What are the different values that X can take? Calculate the probability that X takes the value 5000?

---

In [ ]:
# Sampling space for appointment success (0 corresponds to no sale, 1 corresponds to a sale)
s_appointment = c(0, 1)

# Appointment success and failure probabilities
p_morning = 0.3 # Success probability of sales in morning
p_afternoon =  0.6# Success probability of sales in afternoon
p_appointment = matrix(nrow = 2, ncol = 2, c(1-p_morning, p_morning, 1-p_afternoon, p_afternoon), byrow = TRUE)

# Sampling space for software type
s_software = c(5000, 10000)

# Software type probabilities
p_software = matrix(nrow = 2, ncol = 2, c(0.3, 0.7, 0.5, 0.5), byrow = TRUE)

# Function that simulates one trial of the random experiment which is
# what the salesman earns on a random day
salesResult = function(){
  result = numeric(2)
  # Simulate whether sales happen in morning and afternoon appointments
  for (j in c(1:2)){
    result[j] = sample(s_appointment, size = 1, replace = TRUE, prob = p_appointment[j, ])
  }
  earnings = ifelse(result[1] == 1, sample(s_software, size = 1, replace = TRUE, prob = p_software[1, ]), 0) + ifelse(result[2] == 1, sample(s_software, size = 1, replace = TRUE, prob = p_software[2, ]), 0)
  return(earnings)
}

# Number of simulations
nsimulations = 1e5
simulatedData = replicate(nsimulations, salesResult())

# Probability that the salesman earns Rs 5000
mean(simulatedData == 5000)


# Can we calculate all the probabilities in one shot?
table(simulatedData) / nsimulations


---

**Discrete random variable**: Let ${\color{red}X}$  represent the total earnings from one day which is random; that is, ${\color{red}X}$ is a discrete random variable which can take the values $0, 5000, 10000, 15000, 20000.$ The associated probabilities can be calculated as:

$\begin{align*}P({\color{red}X} = 0)  &= 0.7\times0.4 = 0.28,\\ P({\color{red}X} = 5000) &= 0.3\times0.3\times0.4+0.7\times0.6\times0.5=0.246,\\ P({\color{red}X} = 10000) &= 0.3\times0.3\times0.6\times0.5+0.3\times0.7\times0.4+0.7\times0.6\times0.5=0.321,\\ P({\color{red}X} = 15000) &= 0.3\times0.3\times0.6\times0.5+0.3\times0.7\times0.6\times0.5=0.09,\\ P({\color{red}X} = 20000) &= 0.3\times0.7\times0.4+0.3\times0.6\times0.5 = 0.063.\end{align*}$

---

**Probability Mass Function (PMF)** of the random variable ${\color{red}X}$ is denoted as ${\color{red}{P_X}}(x),$ where $x$ represents the possible values that the random variable ${\color{red}X}$ can take:

$\begin{align*}{\color{red}{P_X}}(0) &= P({\color{red}X} = 0)  = 0.28,\\ {\color{red}{P_X}}(5000) &=P({\color{red}X} = 5000) = 0.246,\\ {\color{red}{P_X}}(10000) &=P({\color{red}X} = 10000) = 0.321,\\ {\color{red}{P_X}}(15000) &=P({\color{red}X} = 15000) = 0.09,\\ {\color{red}{P_X}}(20000) &=P({\color{red}X} = 20000) = 0.063.\end{align*}$

---

---

Plot the frequency of occurences of all possible values of the random variable $\color{red}X$

---

In [ ]:
# Convert simulatedData into a dataframe
dfSales = as.data.frame(simulatedData)

# Add names to the columns
colnames(dfSales) = c('Earning')

p = ggplot(data = dfSales)+
geom_bar(aes(x = factor(Earning)), width = 0.05, fill = 'steelblue')+
  theme(plot.title = element_text(hjust = 0.5, size = 16, face = "bold"),
  axis.text = element_text(size = 12),
  axis.text.x = element_text(size = 14),
  axis.text.y = element_text(size = 14),
  axis.title = element_text(size = 14, face = "bold"))+
  labs(x = 'Revenue (Rs.)',
       y = 'Frequency',
       title = 'Frequency of Salesman Revenues')
p

---

Plot the PMF of the random variable $\color{red}X$

---

In [ ]:
# Convert table of simulatedData into a dataframe
dfSales = as.data.frame(table(simulatedData))

# Add names to the columns
colnames(dfSales) = c('Earning', 'Frequency')

# Add a new column with proportion (probability) values
dfSales['Probability'] = dfSales['Frequency'] / nsimulations

#head(dfSales, 5)

p = ggplot(data = dfSales)+
geom_col(aes(x = Earning, y = Probability), width = 0.05, fill = 'steelblue')+
  theme(plot.title = element_text(hjust = 0.5, size = 16, face = "bold"),
  axis.text = element_text(size = 12),
  axis.text.x = element_text(size = 14),
  axis.text.y = element_text(size = 14),
  axis.title = element_text(size = 14, face = "bold"))+
  labs(x = 'Revenue',
       y = 'Count',
       title = 'PMF of Salesman Revenue')
p


What is the salesman's expected earnings? What does it even mean to say "expected earning?" It is a single number denoted as $E[X]$ and referred to as the *expected value of* $X$ which can be calculated as follows:


---



*   *From a simulation perspective*, it is simply an average of all the simulated earnings:
$\begin{align*}E[X] &\approx \frac{1}{\text{nsimulations}}\times\begin{cases}\text{No. of times 0 appeared}\times0\\\qquad\qquad\quad+\\\text{No. of times 5000 appeared}\times 5000\\\qquad\qquad\quad+\\\text{No. of times 10000 appeared}\times10000\\\qquad\qquad\quad+\\\text{No. of times 15000 appeared}\times15000\\\qquad\qquad\quad+\\\text{No. of times 20000 appeared}\times20000\end{cases}\\&=0\times\underbrace{\frac{\text{No. of times 0 appeared}}{\text{nsimulations}}}_{\color{green}{\text{approximation to }P(X=0)=P_X(0)}}\\&+5000 \times\underbrace{\frac{\text{No. of times 5000 appeared}}{\text{nsimulations}}}_{\color{green}{\text{approximation to }P(X=5000) = P_X(5000)}}\\&+10000 \times\underbrace{\frac{\text{No. of times 10000 appeared}}{\text{nsimulations}}}_{\color{green}{\text{approximation to }P(X=10000) = P_X(10000)}}\\&+15000 \times\underbrace{\frac{\text{No. of times 15000 appeared}}{\text{nsimulations}}}_{\color{green}{\text{approximation to }P(X=15000)=P_X(15000)}}\\&+20000 \times\underbrace{\frac{\text{No. of times 20000 appeared}}{\text{nsimulations}}}_{\color{green}{\text{approximation to }P(X=20000)=P_X(20000)}}.\end{align*}$


---


*   *From a theoretical perspective*, the expected value of $X$ is defined as the weighted sum of the possible values $X$ can take with the corresponding probabilities as the weights:

$\begin{align*} E[X] &= \sum_xxP_X(x),\ \text{where }x = 0, 5000, 10000, 15000, 20000\\\Rightarrow E[X] &= 0\times P_X(0)+5000\times P_X(5000)+10000\times P_X(10000)+15000\times P_X(15000)+20000\times P_X(20000)\\&=0\times0.28+5000\times0.246+10000\times0.321+15000\times0.09+20000\times0.063 = \text{Rs. }7050.\end{align*}$



---

So, we see that the expected value of $X$ is approximately the long-term average of the simulated (or realized) values of $X.$


In [ ]:
# Expected earnings using simulated values
mean(simulatedData) # long term average of the realizations of X

# Expected earnings using the theoretical definition
x = c(0, 5000, 10000, 15000, 20000)
p = c(0.28, 0.246, 0.321, 0.09, 0.063)
sum(x * p)

How much could the salesman's earnings vary from the expected value of the earning calculated in the previous cell?


---


To answer this question, let's first look at the quantity $X-E[X] = X-7050.$ Note that:


*   this is also a random variable with the possible values $0-7050, 5000-7050, 10000-7050, 15000-7050, 20000-7050$;
*   it can be thought of as the random variable that captures the deviation of $X$ from its expected value;
*  the associated probabilities are still the same as:
\begin{array}{c|c|c}
\hline
\color{red}X&\color{red}{X}-\color{cyan}{E[X]} = \color{red}{X}-\color{cyan}{7050}&\color{magenta}{\text{Probability}}\\
\hline
0 & -7050& 0.28\\
5000 & -2050 & 0.246\\
10000 & 2950 & 0.321\\
15000 & 7950 & 0.09\\
20000 0& 12950 & 0.063
\end{array}
---
Now, consider the quantity $(X-E[X])^2 = (X-7050)^2.$ Note that:


*   this is also a random variable with the possible values $(0-7050)^2, (5000-7050)^2, (10000-7050)^2, (15000-7050)^2, (20000-7050)^2$;
*   it can be thought of as the random variable that captures the squared deviation of $X$ from its expected value;
*  the associated probabilities are still the same as:
\begin{array}{c|c|c}
\hline
\color{red}X&(\color{red}{X}-\color{cyan}{E[X]})^2 = (\color{red}{X}-\color{cyan}{7050})^2&\color{magenta}{\text{Probability}}\\
\hline
0 & (-7050)^2 & 0.28\\
5000 & (-2050)^2 & 0.246\\
10000 & (2950)^2 & 0.321\\
15000 & (7950)^2& 0.09\\
20000 & (12950)^2 & 0.063
\end{array}

---

$\color{green}{Variance}$ of the random variable $X,$ denoted as $\text{var}[X],$ is the expected squared deviation of $X$ from its expected value. In simple terms, variance of $X$ is the expected value of the random variable $(X-E[X])^2.$

$\begin{align*}\text{Var}[X] &= E\left[\underbrace{(X-E[X])^2}_{\text{squared deviation random variable}}\right]\\&=
\sum_x (x-E[X])^2\times P_X(x)\\& = (0-7050)^2\times P_X(0)+(5000-7050)^2\times P_X(5000)+(10000-7050)^2\times P_X(10000)+(15000-7050)^2\times P_X(15000)+(20000-7050)^2\times P_X(20000)\\&=(0-7050)^2\times 0.28+(5000-7050)^2\times 0.246+(10000-7050)^2\times 0.321+(15000-7050)^2\times 0.09+(20000-7050)^2\times 0.063\\&=309375\ \text{Rupees squared}.\end{align*}$

---

$\color{green}{Standard\ deviation}$ of the random variable $X,$ denoted as $\text{SD}[X],$ is defined as the square root of its variance:

$\begin{align*}\text{SD}[X] &= \sqrt{\text{Var}[X]} = \sqrt{E\left[(X-E[X])^2\right]}\\&=\sqrt{(0-7050)^2\times 0.28+(5000-7050)^2\times 0.246+(10000-7050)^2\times 0.321+(15000-7050)^2\times 0.09+(20000-7050)^2\times 0.063}\\&\approx \text{Rs. 5830.7}.\end{align*}$

---

In [ ]:
# Simulated earnings of the salesman
simulatedData

In [ ]:
# Sample mean
mean(simulatedData) # sample mean

print('---------')

# Sample variance of the earnings using simulated values
#simulatedData - mean(simulatedData) # sample deviations vector
#(simulatedData - mean(simulatedData))^2 # sample squared deviations vector
mean((simulatedData - mean(simulatedData))^2) # sample variance calculated using simulation
var(simulatedData) # in-built function for variance

print('---------')

# Standard deviation of the earnings using simulated values
sqrt(mean((simulatedData - mean(simulatedData))^2))
sd(simulatedData) # in-built function for standard deviation

print('---------')

# Variance of the earnings using the thoretical definition
x = c(0, 5000, 10000, 15000, 20000)
p = c(0.28, 0.246, 0.321, 0.09, 0.063)
# sum(p*x) # mean or expected value of the random variable
# x - sum(x*p) # deviations random variable
# (x - sum(x*p))^2 # squared deviations randon variable
sum((x - sum(x*p))^2 * p) # variance of the random variable

print('---------')

# Standard deviation of the earnings using the theoretical definition
sqrt(sum((x - sum(x*p))^2 * p))

---

$ X = \text{heart rate of a random patient}$, which is a random variable. We will assume that the heart rates are symmetrically distributed around 74 BPM, and simulate the heart rates of 100 patients.

---

In [ ]:
nsimulations = 100
simulatedData = rnorm(nsimulations, mean = 74, sd = 6)
print(simulatedData)

# Expected value of heart rates (BPM)
mean(simulatedData)

# Variance of heart rates (BPM^2)
mean((simulatedData - mean(simulatedData))^2)

# Standard deviation of hear rates (BPM)
sqrt(mean((simulatedData - mean(simulatedData))^2))

---

Component plot of the simulated heart rates

---

In [ ]:
dfPatient = as.data.frame(simulatedData)
colnames(dfPatient) = c('HR')
#head(dfPatient)
# Component plot of the simulated values (which can be seen as a vector)
p = ggplot(data = dfPatient) +
  geom_point(aes(x = 1:nrow(dfPatient), y = HR), color = 'blue') +
  labs( x = 'Patient #', y = 'HR (BPM)') +
  geom_hline(yintercept = mean(simulatedData), color = 'red') +
  geom_hline(yintercept = mean(simulatedData) + sqrt(mean((simulatedData - mean(simulatedData))^2)), color = 'green') +
  geom_hline(yintercept = mean(simulatedData) - sqrt(mean((simulatedData - mean(simulatedData))^2)), color = 'green') +
  theme(plot.title = element_text(hjust = 0.5, size = 16, face = "bold"),
  axis.text = element_text(size = 12),
  axis.text.x = element_text(size = 14),
  axis.text.y = element_text(size = 14),
  axis.title = element_text(size = 14, face = "bold"))+
  labs(x = 'Patient #',
       y = 'HR (BPM)',
       title = 'Component Plot of Heart Rates (BPM)')
p

---


$\color{orange}{Binomial\ Experiment}$

**Scenario**: a box has 10 balls of which 4 are white and 6 are black balls. We randomly draw 5 balls $\color{green}{with\ replacement}.$ We are interested in the probability that 3 of the 5 drawn balls are white. This is the $\color{green}{binomial\ experiment}$.

Let $X$ be the random variable that represents the number of white balls drawn. Note the two parameters of the experiment:

1. the number of trails or the sample size $n=5$
2. the success proportion in the sample $p=\text{no. of white balls}/\text{total no. of balls} = 4/10.$

We denote the random variable as $$X\sim\text{Bin}(n=5, p = 4/10),$$

and we are interested in $P(X = 3).$

**Probability mass function (PMF)**: $$\begin{align*}P_X(j)&= P(X = j) =nC_j\times p^j\times(1-p)^{n-j}\\\Rightarrow P_X(3) =& P(X = 3) = 5C_3\times (4/10)^3\times(1-4/10)^{5-3}.\end{align*}$$

What is the probability that we will get at most 3 white balls?

**Cumulative distribution function (CDF)**: $$\begin{align*}F_X(j) &= P(X\leq j)\\\Rightarrow F_X(3) &= P(X\leq 3) = P(X=0) + P(X=1)+ P(X=2) + P(X=3).\end{align*}$$

What is the probability that we will get at least 3 white balls?

$$P(X >= 3) = 1 - P(X < 3) = 1 - P(X\leq2).$$

---

In [ ]:
## Calculating the binomial probabilities directly using the functions
## dbinom() and pbinom()
n = 5
p = 4/10
j = 3
dbinom(j, n, p) # P(X = j) where X~Bin(n, p)
dbinom(0, n, p) + dbinom(1, n, p) + dbinom(2, n, p)  + + dbinom(2, n, p) # P(X <= 3)
sum(dbinom(0:j, n, p)) # same as above P(X <= 3)
pbinom(j, n, p) # P(X <= j)
1 - pbinom(2, n, p)  # P(X >= 3) = 1 - P(X < 3) = 1 - P(X <= 2)

In [ ]:
## Calculating the binomial probabilities using simulation with rbinom() function
nsimulations = 1e5
simulatedData = rbinom(nsimulations, n, p) # simulating a binomial random variable
mean(simulatedData == 3) # simulated probability that X = 3
dbinom(3, n, p) # theoretical probability that X = 3
print('-----');
mean(simulatedData <= 2) # simulated probability that X <= 2
pbinom(2, n, p) # theoretical probability that X <= 2
print('-----');
mean(simulatedData > 2) # simulated probability that X > 2
1 - pbinom(2, n, p) # theoretical probability that X > 2

---

Simulating a binomial random variable and plotting its PDF and CDF using the simulated values

---

In [ ]:
# Simulating a binomial random variable
n = 4
p = 0.4
nsimulations = 1e5
simulatedData = rbinom(?, ?, ?)

# Convert simulatedData into a dataframe
dfBinom = as.data.frame(table(?))

# Add names to the columns
colnames(dfBinom) = c('Values', 'Frequency')

# Add a new column with proportion (probability) values
dfBinom['Probability'] = dfBinom[?] / ?

# Add a new column with the cumulative probabilities
dfBinom['Cumprob'] = cumsum(?)

# Plot the PMF of the binomal random variable using the simulated values
p = ggplot(data = ?)+
geom_?(aes(x = ?, y = ?), width = 0.05, fill = 'steelblue')+
  theme(plot.title = element_text(hjust = 0.5, size = 16, face = "bold"),
  axis.text = element_text(size = 12),
  axis.text.x = element_text(size = 14),
  axis.text.y = element_text(size = 14),
  axis.title = element_text(size = 14, face = "bold"))+
  labs(x = 'j',
       y = 'P(X = j)',
       title = 'PMF of X~Bin(n = 4, p = 0.4) using simulated values')
p

# Plot the CDF of the binomal random variable using the simulated values
p = ggplot(data = ?)+
geom_?(aes(x = ?, y = ?), width = 0.05, fill = 'steelblue')+
  theme(plot.title = element_text(hjust = 0.5, size = 16, face = "bold"),
  axis.text = element_text(size = 12),
  axis.text.x = element_text(size = 14),
  axis.text.y = element_text(size = 14),
  axis.title = element_text(size = 14, face = "bold"))+
  labs(x = 'j',
       y = 'P(X = j)',
       title = 'CDF of X~Bin(n = 4, p = 0.4) using simulated values')
p

---

$\color{red}{\textbf{Complete this}}$

**Scenario**: the $\color{green}{binomial\ experiment}$ explained above can be abstracted as a box containing 6 white balls and 4 black balls such that we sample $\color{green}{with\ replacement}$ one ball from the box and repeat it 4 times. Here we calculate the probability that we get 3 white balls by simulating this random experiment.

---


In [ ]:
# A demo of the grepl() function
data = c('w1', 'w2', 'b1', 'b2')
sum(grepl('w.*', data))

In [ ]:
## Sampling space
s = c('w1', 'w2', 'w3', 'w4', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6')

## Probability for sampling an element from the sampling space
p = (1/length(s))* replicate(length(s), 1)

## Simulate the random experiment of drawing 4 balls from 10 balls with
## replacement
nsimulations = 1e1
samplesize = 5
simulatedData = replicate(nsimulations, sample(s, size = samplesize, replace = TRUE, prob = p))

## Check event if 3 of the 5 drawn balls are white
checkEvent = function(data){
  if (? == 3){
    return(1)
  }
  else{
    return(0)
  }
}
#simulatedData
## Calculate the probability that 3 of the 5 drawn balls are white
mean(apply(?, ?, ?))

## Calculate probability that 3 of the 5 drawn balls are white using dbinom()
dbinom(?, ?, ?) # P(X = 3) when X~Bin(n = 5, p = 0.4)

In [ ]:
simulatedData

---

$\color{red}{\textbf{Complete this}}$

**Scenario**: the $\color{green}{binomial\ experiment}$ explained above can be abstracted as a box containing a white and black ball such that we sample $\color{green}{with\ replacement}$ one ball from the box and repeat it 4 times. Here, the probability of drawing a white ball is 4/10 and probability of drawing a black ball is 6/10.

---


In [ ]:
## Sampling space
s = c('w', 'b')

## Probability for sampling an element from the sampling space
p = c(?, ?)

## Simulate the random experiment of drawing 4 balls from 10 balls with
## replacement
nsimulations = 1e5
samplesize = ?
simulatedData = replicate(nsimulations, sample(s, size = ?, replace = TRUE, prob = ?))

# Check event if 3 of the 5 drawn balls are white
checkEvent = function(data){
  if (sum(data == ?) == ?){
    return(?)
  }
  else{
    return(?)
  }
}

## Calculate the probability that 3 of the 5 drawn balls are white
mean(apply(?, ?, ?))

$\color{red}{\textbf{Complete this}}$

---

**Example:** an oil company has a $20\%$ chance of striking oil when drilling a well. What is the probability the company will strike oil 3 times in 7 drilling attempts?

$$X\sim\text{Bin}(n = ?, p = 0.?)$$

and we are interested in $P(X = ?).$

---

$\color{red}{\textbf{Complete this}}$

---

**Example**: approximately 42% of people have type O blood. On a given day in a blood bank, 120 people arrive to donate blood. What is the probability that 30 of those 120 people have type O blood?

$$X\sim\text{Bin}(n = ?, p = ?),$$

We want $P(X=\,?).$

---

$\color{red}{\textbf{Complete this}}$

---

**Example**: A certain stoplight, when coming from the North, is green approximately 31% of
the time. Over the next few days, someone comes to this light 8 times from the
North. We are interested in the probability that the person will come across green light 5 times.

$$X\sim\text{Bin}(n = ?, p = ?).$$

We want $P(X=\,?).$

---

$\color{red}{\textbf{Complete this}}$

---

**Example**: A certain stoplight, when coming from the North, is green approximately 31% of
the time. Over the next few days, someone comes to this light 8 times from the
North. We are interested in the probability that the person will come across green light at least 5 times.

$$X\sim\text{Bin}(n = ?, p = ?).$$

We want $P(X\geq\,?).$

---

$\color{red}{\textbf{Complete this}}$

---

**Example**: A certain stoplight, when coming from the North, is green approximately 31% of
the time. Over the next few days, someone comes to this light 8 times from the
North. We are interested in the probability that the person will come across green light at most 5 times.

$$X\sim\text{Bin}(n = ?, p = ?).$$

We want $P(X\leq\,?).$

---

$\color{red}{\textbf{Complete this}}$

---

**Example**: It is known that diskettes produced by a certain company will be defective with probability .01, independently of each other. The company sells the diskettes in packages of size 10 and offers a money-back guarantee that at most 1 of the 10 diskettes in the package will be defective. The guarantee is that the customer can return the entire package of diskettes if he or she finds more than one defective diskette in it. If someone buys 3 packages, what is the probability that he or she will return exactly 1 of them?

Let $X$ be the number of packages that are returned.

$$X \sim\text{Bin}(n = 3, p_X =\ ?),$$

and we want $P(X =\,?).$

The number of defective items in a package:

$$Z \sim\text{Bin}(n = ?, p_Z = ?).$$

$$p_X = P(Z >\,?) = 1-P(Z <=\,?).$$

---

---

**Example**: a machine produces items in batches. For each batch, the machine can be in control or out of control. Suppose the machine is in control for 99\% of the production batches; The production defect rate is 0.01 when the machine is in control and 0.1 when the machine is out of control. Assume that the production size is 1000 items per batch.

Suppose we want to update our *prior* knowledge of the probability that the machine is in/out-of control. To that end, we select a small random sample of 5 items from a production batch and inspect how many among them are defective, which is represented by $X$. Compute the missing entries in the following table using the hint given below:

---

![table](https://dsm04pap002files.storage.live.com/y4mXfEFb-HaPGZNdqxBRlJdFnJejZZ28kRYoGI5acBdY0WuKLxVS6y_whP6zmLpGxMG7_mQYC2OIPa2wkouJ0ofqAiw_6Nh-6wT45EJpEPeEsmzJ2wCj310cGg-zvZS4NEAY6OaeQrEKXTNThqxHIZVfx8m5E5nNY-NxfX1KBqOkbkNrNGMIZQENpUieuQRTwq1?width=660&height=178&cropmode=none)

---

*Hint*:

![hint](https://dsm04pap002files.storage.live.com/y4mYL8V8jmlmzvzkuSs6KxX06nfgHDMNCkbgSMsV2E2edsy31NRHdfdsvm8bG7Y7EMh7PuKO-zxHYUMpYT2veBXxAytwCKgOZeJG413vD8p7LXnVIzdOsForszztScSBh8LCLhxYrrf-w-g6BmDi8EqXz18T-PuNMCuK9qzVhJO2L77oBH5hEXkHC2hNaLBkozG?width=660&height=167&cropmode=none)

---

In [ ]:
n = 5
defectRate_IC = 0.01 # defect rate when machine is in control
defectRate_OC = ? # defect rate when machine is out of control
p_IC = ? # prior probability that the machine is in control
p_OC = 1-p_IC # prior probability that the machine is out of control
j = ?:?

# Posterior probability that the machine is in control after
# observing j defective items in a sample of 5 items chosen
# from a batch
(dbinom(?, ?, ?) * p_IC) / (dbinom(j, n, defectRate_IC) * ? + dbinom(?, ?, ?) * ?)

---

$\color{orange}{Hypergeometric\ Experiment}$


**Scenario**: a box has 4 white and 6 black balls. We randomly draw 5 balls $\color{green}{without\ replacement}.$ We are interested in the probability that 3 of the 5 drawn balls are white. This is the $\color{green}{hypergeometric\ experiment}$.

Let $X$ be the random variable that represents the number of white balls drawn. Note the two parameters of the experiment:

1. the number of successes $n_s=\text{no. of white balls}=4.$
2. the number of failures $n_f = \text{no. of black balls}=6.$
3. the sample size $n=5$

We denote the random variable as $$X\sim\text{HypGeom}(n_s = 4, n_f = 6, n = 5),$$

and we are interested in $P(X = 3).$

**Probability mass function (PMF)**: $$\begin{align*}P_X(j)&= P(X = j) =\dfrac{\binom{n_s}{j}\binom{n_f}{n-j}}{\binom{n_s+n_f}{n}}\\\Rightarrow P_X(3) =& P(X = 3) = \dfrac{\binom{4}{3}\binom{6}{5-3}}{\binom{4+6}{5}}.\end{align*}$$

What is the probability that we will get at most 3 white balls?

**Cumulative distribution function (CDF)**: $$\begin{align*}F_X(j) &= P(X\leq j)\\\Rightarrow F_X(3) &= P(X\leq 3) = P(X=0) + P(X=1)+ P(X=2) + P(X=3).\end{align*}$$

What is the probability that we will get at least 3 white balls?

$$P(X >= 3) = 1 - P(X < 3) = 1 - P(X\leq2).$$

---

In [ ]:
n_s = 4
n_f = 10
n = 5
j = 3
dhyper(j, n_s, n_f, n) # P(X = j)
phyper(j, n_s, n_f, n) # P(X <= j)
1-phyper(j-1, n_s, n_f, n) # P(X >= j)

$\color{red}{\textbf{Complete this}}$

---

Simulating a hypergeometric random variable and plotting its PMF and CDf using simulated values.

---

---

How are the binomial and hypergeometric experiments related?

If $X\sim\text{HypGeom}(n_s, n_f, n)$ such that *both* the the following conditions hold:
1. sample size is much smaller than population size, $n << (n_s+n_f)$, and
2. number of successes $n_s$ is not close to  $0$ and population size $n_s+n_f.$

Then, $P(X = j)$ calculated using the hypergeometric PMF (or $\texttt{dhyper()}$) will result in approximately the same answer as calculating $P(X=j)$ using the binomial PMF (or $\texttt{dbinom()}$) where

$$X\sim\text{Bin}(n, p)\ \text{approximately where }p=n_s/(n_s+n_f).$$

---

In [ ]:
# (1) Condition 1 "sample size is small compared to population size" satisfied
# (2) Condition 2 "success proportion is not too close to 0 or 1" not satisfied
dhyper(38, 9900, 100, 40) # X~HypGeom(ns = 9900, nf = 100, n= 40), P(X = 38)
dbinom(38, 40, 0.99) # X~Bin(n = 40, p = 0.99), P(X = 38)

# (1) Condition 1 "sample size is small compared to population size" not satisfied
# (2) Condition 2 "success proportion is not too close to 0 or 1" satisfied
dhyper(3600, 4000, 6000, 9000) # X~HypGeom(ns = 4000, nf = 6000, n = 9000), P(X = 3600)
dbinom(3600, 9000, 0.4) # X~Bin(n = 9000, p = 0.4), P(X = 3600)

# (1) Condition 1 "sample size is small compared to population size"  satisfied
# (2) Condition 2 "success proportion is not too close to 0 or 1" satisfied
dhyper(40, 4000, 6000, 100)
dbinom(40, 100, 0.4)

---

In a forest that has 1000 tigers, 250 are captured, tagged, and released. A few weeks later, a sample of 10 tigers from the forest is captured. Calculate the
probability that

- exactly half of those captured tigers are tagged
- at most half of those captured tigers are tagged
- at least half of those captured tigers are tagged

---

$\color{red}{\textbf{Complete this}}$


---

**Example:** an oil company has the following record in the past: out of 10000 drilling attempts, it has struck oil 2000 times. What is the probability the company will strike oil 3 times in 7 drilling attempts?

$$X\sim\text{HypGeom}(n_s = ?, n_f = ?, n = ?)$$

and the probability we are interested in is $P(X=\,?).$

---

$\color{red}{\textbf{Complete this}}$

---

A company has a pool of 15 applicants (10 male, 5 female) for a particular position that has 3 current openings. Calculate the probability that none of the
positions are filled by males.

$$X\sim\text{HypGeom}(n_s = ?, n_f = ?, n = ?)$$

and the probability we are interested in is $P(X=\,?).$

---

$\color{red}{\textbf{Complete this}}$

---

**Example:** an assembly line produces products that they put into boxes of 50. The inspector then randomly picks 3 items inside a box to test to see if they are defective. In a box containing 4 defectives, they are interested in the probability that at least one of the three items sampled is defective.

$$X\sim\text{Hyper}(n_s = ?, n_f = ?, n = ?)$$

and the probability we are interested is need $P(X \geq\,?).$

---

$\color{red}{\textbf{Complete this}}$

---

**Example:** an assembly line produces products that they put into boxes of 50. The inspector then randomly picks 3 items inside a box to test to see if they are defective. In a box containing 4 defectives, they are interested in the probability that at most one of the three items sampled is defective.

We need $$ P(X \leq 1),$$ where $$X\sim\text{Hyper}(n_s = ?, n_f = ?, n = ?).$$


---

---


$\color{orange}{Negative\ Binomial\ Experiment}$


**Scenario**: a box has 4 white and 6 black balls. We randomly draw balls $\color{green}{with\ replacement}.$ We are interested in the probability that 4th white ball will be drawn in the 10th attempt. This is the $\color{green}{negative\ binomial\ experiment}.$

---

In [ ]:
## Sampling space
s = c('w', 'b')

## Probability for sampling an element from the sampling space
p = c(4/10, 6/10)

## Simulate the random experiment of drawing 10 balls with replacement
nsimulations = 1e6
simulatedData = replicate(nsimulations, sample(s, size = 10, replace = TRUE, prob = p))

## Check event if 4th white ball is drawn in the 10th attempt
checkEvent = function(data){
  if ((sum(data == 'w') == 4) & (data[10] == 'w')){
    return(1)
  }
  else{
    return(0)
  }
}

## Calculate probability that the 4th white ball is drawn in the 10th attempt
#simulatedData
mean(apply(simulatedData, 2, checkEvent))
dnbinom(10-4, 4, 0.4) # p = 0.4, r = 4
# The probability that "we get the 4th success in the 10th attempt with a
# success probability per trail of 0.4" is equivalent to the probability that
# "we have 10-4 failures before the 4th sucess with a sucess probability
# of 0.4 per trial."


---

**Example:** an oil company has a $20\%$ chance of striking oil when drilling a well. What is the probability the company drills $7$ times to strike oil the 3rd time?

$$X\sim\text{NegBin}(r = 3, p = 0.2)$$ and we are interested in $P(X=7).$

---

In [ ]:
r = 3
p = 0.2
j = 7
dnbinom(j-r, r, p)
choose(j-1, r-1)*(p^r)*(1-p)^(j-r)

$\color{red}{\textbf{Complete this}}$

---
  
**Example:** a person conducting telephone surveys must get 4 more completed surveys before their job is finished. On each randomly dialed number, there is a 90% chance of the participant rejecting the call. What is the probability that the person will finish their job at the 10th call?

$$X\sim\text{NegBin}(r = ?, p = ?)$$

and we are interested in $P(X = \,?).$
  
---

$\color{red}{\textbf{Complete this}}$

---

**Example**: at an airport, it is know that approximately 2 out of 10 passengers have a metallic object. If left undetected at the manual security check at the airport entrance,  such a metallic object will raise an alarm when the passenger walks through an automated screening machine. It is considered a security breach when the alarm gets raised 20 times a day. What is the probability of a security breach on a particular day when the 100th passenger walks through the automated screening machine?

$$X\sim\text{NegBin}(r = ?, p = ?)$$

and we are interested in $P(X=\,?).$

---

$\color{red}{\textbf{Complete this}}$

---

The following questions are designed to test your understanding of which discrete random variable you would use for the modeling and calculation purposes:

1. Suppose a production line has a 20% defective rate.
- What is the probability that the first defective will occur in the 5th inspection?
- What is the probability that it will take more than 10 inspections to identify the first defective?
- Suppose that the first 10 inspections have been found to be free of defectives. What is the probability that the first defective will occur during the 15th inspection? **Hint**: apply Bayes’ theorem to $P(X = 15\,|\,X > 10).$
2. Microprocessors for an electronic device are procured from two factories $A$ and $B$ and sent as a single shipment. 70% of the microprocessors come from $A$ and the remaining from $B.$ Suppose that a microprocessor manufactured in $A$ is twice as likely to be defective as one manufactured in $B.$ A shipment containing 10,000 microprocessors arrives of which 200 are found to be defective. Approximately how many defective microprocessors from this shipment can be attributed to $A?$

$\color{red}{\textbf{Complete this}}$


---

This is an exercise on simulating a generic random variable like the salesman's earnings random variable:

To determine whether they have bird flu, 50 people have their blood tested. However, rather than testing each individual separately, it is decided to first place the people into groups of 5. The blood samples of the 5 people in each group will be pooled and analyzed together. If the test is negative, one test will suffice for the 5 people, whereas if the test is positive, each of the 5 people will also be individually tested and, in all, 6 tests will be
made on this group. Note that we assume that the pooled test will be positive if at least one person in the pool has bird flu. Assume that the probability that a person has bird flu is 15% for all people, independently of each other.

Let $X$ be the random variable representing the number of tests required for a random group of 5 people.

- What are the values that $X$ can take?
- Calculate the associated probabilities using simulation and confirm your answers with pen & paper probabilities.
- Compute the expected number of tests for a random group of 5 people if we adopt the testing strategy proposed above using simulation first and then confirm your answer using pen & paper.

---